In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import string
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec25.txt'

In [2]:
good_items = """\
shell
fuel cell
fixed point
polygon
antenna
candy cane
hologram""".split('\n')

steps = """\
south
north
west
west
west
east
east
east
north
west
west
west
east
east
east
north
north
south
west
west""".split('\n')

def _parse_items_here(text):
    collection = []
    start_collecting = False
    for line in text.split('\n'):
        if start_collecting:
            if line:
                collection.append(line.strip()[2:])
            else:
                return collection

        if line.startswith('Items here:'):
            start_collecting = True
    return collection

def powerset(lst):
    output = [[]]
    for x in lst:
        output.extend([out + [x] for out in output])
    return output


class IOWrapper:
    def __init__(
            self,
            steps: Optional[List[str]] = None,
            good_items: Optional[List[str]] = None,
            mode: str = 'manual'
        ):
        self.good_items = set(good_items)
        self.steps = deque(steps) if steps else deque([])
        self.last_output = []

        self.queue = deque([])
        self.combos_tried = 0
        self.pset = powerset(self.good_items)[1:]  # Drop empty combo
        
        if mode == 'automatic':
            self.mode = 'pickup'
        else:
            self.mode = 'manual'
        

    def __call__(self, val):
        self.last_output.append(chr(val))
        print(chr(val), end='')
    
    def command(self, *commands):
        print('Command: ', end='')
        for command in commands:
            print(command, end='')
            for c in command:
                self.queue.append(ord(c))
        self.queue.append(ord('\n'))
        print()

    def get(self):
        last_output = ''.join(self.last_output)
        while not self.queue:
            if self.mode == 'manual':
                print(last_output)
                val = input('Input? ')
                self.command(val)

            elif self.mode == 'pickup':
                self.last_output = []
                items_here = _parse_items_here(last_output)
                while items_here:
                    item = items_here.pop()
                    if item in good_items:
                        self.command('take ', item)
                if self.steps:
                    self.command(self.steps.popleft())
                else:
                    self.mode = 'drop'
                    
            elif self.mode == 'drop':
                if '== Security Checkpoint ==' not in last_output:
                    self.mode = 'manual'
                    continue

                for item in self.good_items:
                    self.command('drop ', item)
            
                self.mode = 'try_combo'
            
            elif self.mode == 'try_combo':
                self.command('take ', 'polygon')
                self.command('take ', 'shell')
                self.command('take ', 'candy cane')
                self.command('take ', 'fixed point')
                self.mode = 'manual'
                
                # The following code will try all combos. but we'll speed
                # it up by knowing the answer
                # combo = self.pset[self.combos_tried]
                # self.combos_tried += 1
                # for item in combo:
                #     self.command('take ', item)
                # self.mode = 'drop'
                self.command('west')
        return self.queue.popleft()

In [3]:
ops = sim.read_ops(INPUT_PATH.read_text())

In [4]:
io_wrapper = IOWrapper(steps=steps, good_items=good_items, mode='automatic')
robot = sim.Computer(ops, inputs=io_wrapper, output_func=io_wrapper)

In [5]:
robot.simulate()




== Hull Breach ==
You got in through a hole in the floor here. To keep your ship from also freezing, the hole has been sealed.

Doors here lead:
- north
- south
- west

Command?
Command: south



== Gift Wrapping Center ==
How else do you wrap presents on the go?

Doors here lead:
- north

Items here:
- fixed point

Command?
Command: take fixed point
Command: north

You take the fixed point.

Command?



== Hull Breach ==
You got in through a hole in the floor here. To keep your ship from also freezing, the hole has been sealed.

Doors here lead:
- north
- south
- west

Command?
Command: west



== Sick Bay ==
Supports both Red-Nosed Reindeer medicine and regular reindeer medicine.

Doors here lead:
- east
- west

Command?
Command: west



== Observatory ==
There are a few telescopes; they're all bolted down, though.

Doors here lead:
- north
- east
- west

Items here:
- infinite loop

Command?
Command: west



== Stables ==
Reindeer-sized. They're all empty.

Doors here lead:
- eas

-1